In [167]:
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
import math
import operator
import cv2

image functions

In [168]:
def loadImage(path, name):
    image = cv2.imread('{path}/{name}'.format(path=path,name=name))
    image = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
    image = cv2.resize(image, (400,300), interpolation = cv2.INTER_AREA)
    image = 255 - image
    image = np.transpose(image)
    return np.asarray(image)

def saveImage(name, image, invert = True):
    image = np.transpose(image)
    if invert:
        cv2.imwrite('./img/' + name, 255 - image)
    else:
        cv2.imwrite('./img/' + name, image)

def thinnImage(image):
    image = cv2.ximgproc.thinning(image)
    return image

load image and generate binary image

In [169]:
image = loadImage("./../src/testImages","1.png")
(thresh, binary) = cv2.threshold(image, 127, 255, cv2.THRESH_BINARY)

saveImage("image.png", image)
saveImage("binary.png", binary)

filter functions

In [170]:
def getRegion(size, pos, image):
    xOffset = math.floor(size[0]/2)
    yOffset = math.floor(size[1]/2)
    return image[pos[0]-xOffset:pos[0]+xOffset+1, pos[1]-yOffset:pos[1]+yOffset+1]

def equal(filter, img):
    for x in range(0, len(filter)):
        for y in range(0, len(filter[0])):
            if filter[x][y] != -1 and filter[x][y] != img[x][y]:
                return False
    return True

def applyFilterOnPoints(filter, image, points):
    size = (len(filter),len(filter[0]))
    features = []
    for (x, y) in points:
        if equal(filter, getRegion(size, (x, y), image)):
            features.append((x,y))
    return features

def rotateFilter(filter):
    rotatedFilters = []
    for k in range(0,4):
        rotatedFilters.append(np.rot90(filter, k))
    return rotatedFilters

get Black pixel

In [171]:
points = []
for x in range(0, len(binary)):
    for y in range(0, len(binary[x])):
        if binary[x][y] == 255:
            points.append((x,y))

remove black sections

In [172]:
filter = np.full((13, 13), 255)
features = applyFilterOnPoints(filter, binary, points)

for (x,y) in features:
    binary[x][y] = 0

saveImage("ohneSpulen.png", binary)

thinn image

In [173]:
thinn = thinnImage(binary)

saveImage("thinn.png", thinn)

replace certain features

In [174]:
cross = [[0,255,0],[255,255,255],[0,255,0]]
tcross = rotateFilter([[-1,0,-1],[255,255,255],[0,255,0]])
corner = rotateFilter([[-1,-1,-1],[-1,255,255],[-1,255,0]])

crossFeatures = applyFilterOnPoints(cross, thinn, points)

for (x,y) in crossFeatures:
    thinn[x][y] = 0
    thinn[x][y-1] = 0
    thinn[x-1][y-1] = 255

tcrossFeatures = []
for filter in tcross:
    tcrossFeatures += applyFilterOnPoints(filter, thinn, points)

for (x,y) in tcrossFeatures:
    thinn[x][y] = 0

for filter in corner:
    cornerFeatures = applyFilterOnPoints(filter, thinn, points)
    for (x,y) in cornerFeatures:
        thinn[x][y] = 0



saveImage("modifiedThinn.png", thinn)